In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
ENV='local' 
# ENV='colab'

In [3]:
if 'colab' in ENV:
    !sudo apt-get install sox
    !pip install opensmile

In [4]:
import sys
sys.path.append('../src')

import pandas as pd
from data import Dataset

### Dataset

Creamos el dataset: Este ya se encarga de bajar los datos por unica vez.

In [15]:
speech_dataset = Dataset(dataset_name='speech')
song_dataset   = Dataset(dataset_name='song')

Dataset alredy downloaded!
Dataset alredy downloaded!


Le pedimos que nos devuelve todos los ejemplo del dataset:

In [7]:
speech_examples = speech_dataset.get_all()

100%|██████████| 1440/1440 [01:58<00:00, 12.15it/s]


In [8]:
song_examples = song_dataset.get_all()

100%|██████████| 1012/1012 [01:47<00:00,  9.37it/s]


In [9]:
speech_examples.head()

,actor,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,file,start,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,23,./speech/Actor_23/03-01-04-02-01-01-23.wav,03,01,04,02,01,01,./speech/Actor_23/03-01-04-02-01-01-23.wav,0 days,...,0.044101,0.007548,0.015374,1.877934,1.425178,0.251667,0.247482,0.367143,0.419309,-39.951572
1,23,./speech/Actor_23/03-01-04-02-02-02-23.wav,03,01,04,02,02,02,./speech/Actor_23/03-01-04-02-02-02-23.wav,0 days,...,0.011092,0.008020,0.014090,1.694915,0.856531,0.357500,0.072241,0.634000,0.483429,-42.310856
2,23,./speech/Actor_23/03-01-06-02-01-02-23.wav,03,01,06,02,01,02,./speech/Actor_23/03-01-06-02-01-02-23.wav,0 days,...,0.101003,0.014311,0.014765,2.339181,0.593472,0.635000,0.335000,0.683333,0.449024,-34.312801
3,23,./speech/Actor_23/03-01-07-01-01-02-23.wav,03,01,07,01,01,02,./speech/Actor_23/03-01-07-01-01-02-23.wav,0 days,...,0.062009,0.011443,0.021647,2.528090,1.424501,0.180000,0.099800,0.416667,0.422834,-44.573288
4,23,./speech/Actor_23/03-01-03-01-01-02-23.wav,03,01,03,01,01,02,./speech/Actor_23/03-01-03-01-01-02-23.wav,0 days,...,0.032799,0.011666,0.009868,2.153846,0.625000,0.570000,0.320000,0.670000,0.452548,-36.173256


In [10]:
song_examples.head()

,actor,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,file,start,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,23,./song/Actor_23/03-02-04-02-01-01-23.wav,03,02,04,02,01,01,./song/Actor_23/03-02-04-02-01-01-23.wav,0 days,...,0.087133,0.016804,0.010952,1.405975,0.709220,0.865000,0.985051,0.690000,0.452180,-34.619511
1,23,./song/Actor_23/03-02-06-01-02-02-23.wav,03,02,06,01,02,02,./song/Actor_23/03-02-06-01-02-02-23.wav,0 days,...,0.035799,0.012916,0.023227,1.931331,0.650759,0.806667,0.285346,0.703333,0.378535,-31.953684
2,23,./song/Actor_23/03-02-06-01-01-01-23.wav,03,02,06,01,01,01,./song/Actor_23/03-02-06-01-01-01-23.wav,0 days,...,0.041038,0.012036,0.028251,2.433628,1.342282,0.368333,0.246537,0.304286,0.415343,-36.003284
3,23,./song/Actor_23/03-02-02-01-01-01-23.wav,03,02,02,01,01,01,./song/Actor_23/03-02-02-01-01-01-23.wav,0 days,...,0.001443,0.009786,0.005541,1.492537,0.646552,0.826667,0.655862,0.522500,0.476360,-39.769188
4,23,./song/Actor_23/03-02-02-02-01-01-23.wav,03,02,02,02,01,01,./song/Actor_23/03-02-02-02-01-01-23.wav,0 days,...,0.012821,0.011602,0.006391,1.503760,0.569260,1.010000,0.800875,0.542500,0.461919,-39.360474


In [11]:
speech_examples['audio_type'] = 'speech'
song_examples['audio_type'] = 'song'

In [12]:
dataset = pd.concat([speech_examples, song_examples])

In [13]:
dataset.to_csv('dataset.csv')

In [14]:
dataset.head()

,actor,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,file,start,...,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,audio_type
0,23,./speech/Actor_23/03-01-04-02-01-01-23.wav,03,01,04,02,01,01,./speech/Actor_23/03-01-04-02-01-01-23.wav,0 days,...,0.007548,0.015374,1.877934,1.425178,0.251667,0.247482,0.367143,0.419309,-39.951572,speech
1,23,./speech/Actor_23/03-01-04-02-02-02-23.wav,03,01,04,02,02,02,./speech/Actor_23/03-01-04-02-02-02-23.wav,0 days,...,0.008020,0.014090,1.694915,0.856531,0.357500,0.072241,0.634000,0.483429,-42.310856,speech
2,23,./speech/Actor_23/03-01-06-02-01-02-23.wav,03,01,06,02,01,02,./speech/Actor_23/03-01-06-02-01-02-23.wav,0 days,...,0.014311,0.014765,2.339181,0.593472,0.635000,0.335000,0.683333,0.449024,-34.312801,speech
3,23,./speech/Actor_23/03-01-07-01-01-02-23.wav,03,01,07,01,01,02,./speech/Actor_23/03-01-07-01-01-02-23.wav,0 days,...,0.011443,0.021647,2.528090,1.424501,0.180000,0.099800,0.416667,0.422834,-44.573288,speech
4,23,./speech/Actor_23/03-01-03-01-01-02-23.wav,03,01,03,01,01,02,./speech/Actor_23/03-01-03-01-01-02-23.wav,0 days,...,0.011666,0.009868,2.153846,0.625000,0.570000,0.320000,0.670000,0.452548,-36.173256,speech
